Lets import some of the libraries immediately required.

In [ ]:
import torch
import os
import re

import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image

import torchvision


https://www.youtube.com/watch?v=2zptQGC-pHY&list=PLhhyoLH6IjfxkVb3Yx4mjLdwMWP5S_Hct&index=2

# Is data accessible ?

Since the dataset provided is zippped, lets unzip it first.

In [ ]:
!unzip -qo ../input/dogs-vs-cats-redux-kernels-edition/test.zip
!unzip -qo ../input/dogs-vs-cats-redux-kernels-edition/train.zip

Enumerating the directory we see that it has unzipped

In [ ]:
!ls

# Prepping the data for the model

lets create a transform. A transform does exactly as it sounds it transforms the image to a form, in this case since we want the images to be of uniform size we resize it, then convert it into a tensor followed by normalise. The values I copied from other kernel. Its basically something you do.

In [ ]:
import torchvision.transforms as transforms

my_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229,0.224,0.225],
    ),
   
])

lets look at what is actually in the train directory, the file names have their labels in the name itself. here are 5 of the files in train directory

In [ ]:
os.listdir("./train")[:5]

Lets just check whether our transform works on individual images

In [ ]:
img = Image.open("./train/cat.10366.jpg")
img = my_transform(img)

## The pytorch dataset and dataloader

Okay now we create the dataset class. All dataset classes are supposed to have three functions, __init__ (initialiser) , __len__ (returns length of dataset) and __getitem__ (to get individual items), basically if you implement Dataset from pytorch with these three functions, you hava a pytorch Dataset.

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, rootdir, transform=my_transform, partial_data=None):
        self.rootdir = rootdir
        self.transform = transform
        if partial_data is not None:
            self.images = os.listdir(rootdir)[:partial_data]
        else :
            self.images = os.listdir(rootdir)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        
#         print(os.path.join(self.rootdir,self.images[index]))
        img = Image.open(os.path.join(self.rootdir,self.images[index])).convert("RGB")
        
#         image = np.array(img)
        
        if self.transform:
            img = self.transform(img)
        
        if "cat" in self.images[index]:
            label = 0
            
        elif "dog" in self.images[index]:
            label = 1
        else :
            label = -1
            
        return img, label

lets intiatiate our dataset class.

In [ ]:
train_dataset = CatsDogsDataset(rootdir="./train", partial_data=5000) 
# train_dataset = CatsDogsDataset(rootdir="./train")

In [ ]:
train_dataset.__len__()

is it working?

In [ ]:
x, y = train_dataset[100]
x.shape, y

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x.permute(1,2,0))

batch size decides how many of your data items are taken together in a single batch, while you apply the back propagation.

### Hyperparameters

In [ ]:
batch_size = 20
learning_rate = 1e-4 #0.001
num_epochs = 1
num_workers = 4
pin_memory =True
checkpoint_file = "checkpoint.tar"
weight_decay = 1e-4


Lets get the dataloader

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

# Okay. Now onto the model,

In [ ]:
from torchvision.models import resnet18

In [ ]:
model = resnet18(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
import torch.nn as nn

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = model.to(device=device)

In [ ]:
model

In [ ]:
loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [ ]:
from tqdm import tqdm

for epoch in range(num_epochs):
    for data, target in tqdm(train_dataloader):
        data = data.to(device=device)
        target = target.to(device=device)
        
        score = model(data)
        
        loss = loss_criterion(score, target)
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
    print(f"For epoch: {epoch} loss: {loss}")
        

In [ ]:
def check_accuracy(model, loader):
    model.train()
    
    total_correct = 0
    total_predictions = 0
    
    for x, y in tqdm(loader):
        x = x.to(device=device)
        y = y.to(device=device)
        
        
        score = model(x)
        z, predictions = score.max(1)
        
#         print(f"score shape : {score.shape} , z: {z}, predictions: {predictions}, predictions shape: {predictions.shape}")
        
        total_correct = (y==predictions).sum()
        total_predictions += predictions.shape[0]
    
    print(f"Out of {total_predictions}, total correct: {total_correct} with an accuracy of {float(total_correct/total_predictions)* 100}")

In [ ]:
check_accuracy(model, train_dataloader)

That is bad performance , but at least it was quick, lets work on inference.

# Inference!

In [ ]:
import pandas as pd

submission = pd.read_csv('../input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
submission.head()

In [ ]:
class testDataset(Dataset):
    def __init__(self, rootdir, transform=my_transform, partial_data=None):
        self.rootdir = rootdir
        self.transform = transform
        if partial_data is not None:
            self.images = os.listdir(rootdir)[:partial_data]
        else :
            self.images = os.listdir(rootdir)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        
#         print(os.path.join(self.rootdir,self.images[index]))
        img = Image.open(os.path.join(self.rootdir,self.images[index])).convert("RGB")
        
#         image = np.array(img)
        
        if self.transform:
            img = self.transform(img)
        
#         if "cat" in self.images[index]:
#             label = 0
            
#         elif "dog" in self.images[index]:
#             label = 1
#         else :
#             label = -1
            
        return img

In [ ]:
test_dataset = testDataset("./test")

In [ ]:
len(submission), test_dataset.__len__()

In [ ]:
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:


predictions = []
for x in tqdm(test_dataloader):
    x = x.to(device=device)
    
    score = model(x)
    _, prediction = score.max(1)
    
    predictions.append(prediction)
    

In [ ]:
len(predictions)

In [ ]:
submission["label"] = [int(x) for x in predictions]
submission.head()

In [ ]:
submission.to_csv("my_submission.csv", index=False)